We want to use the model and algorithm of Tutorial--Van der Pol's dynamics with spike train observations.ipynb on the data in the folder Data.

In [1]:
# Libraries
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
import sys

# .py
sys.path.append("..")
from inference import GaussMarkovLagrange
from likelihoods import PointProcess
from mappings import AffineMapping
from transition import FixedPointSparseGP, SparseGP
from kernels import RBF
from linkfunctions import Exp
from models import PointProcessGPSDEmodel, GPSDE
sys.path.append("../Data")
from Load_plot_data import load_neuron_data, roster_plot

# Load the data and modify its format

In [2]:
#Load data
ids,times=load_neuron_data('../Data/Cellline1_Date190328_Chip2135.npz')
N=np.max(ids) # Number of neurons
trLen=np.array([np.max(times)]) #Tmax must be an array to have one for each trial

The data from the files is of the form
( id : neurons ids,
times : times of spike )

For the model we need a list of array where array i contains the times at which neuron i had a spike

We can observe using unique and some tests that the ids are all the numbers between 0 and 1016 except 131 and 899. We suppose that they are not present beacause they didn't have a spike.

In [3]:
Yspike=[]
for i in range(N):
    Yspike.append(times[ids==i])
print(len(Yspike))

#For now there is only one trial, to change if we want to define trials
Yspike=[Yspike]

1016


# Choices for the model

In [4]:
#Choice of variables
dtgrid = 0.004 # discretisation for solving ODEs => maybe try to use one from the data instead
xDim = 2 # two latents

In [5]:
#Choice of model components
link = Exp() # exponential link function (to define nonlinearity in likelihood )
like = PointProcess(Yspike, link, trLen, dtstep=dtgrid, nLeg=100) # point process likelihood 
kern = RBF(xDim) # RBF kernel

#To initialize the Affine mapping from latent to observations
C = 2.*np.random.rand(xDim,N) * np.random.choice([-1,1],size=(xDim,N)) 
d = 0.1*np.random.randn(1,N)
outputMapping = AffineMapping(torch.tensor(C), torch.tensor(d)) # affine output mapping, used inn the forward of the model

In [6]:
#Choose the inducing points and transition function

# generater inducing point locations on a 2D grid (for sparse gp)
xmin, xmax = -2., 2.
Zs1, Zs2 = torch.meshgrid([torch.linspace(xmin,xmax,5), torch.linspace(xmin,xmax,5)])
numZ = 25
Zs = torch.cat((Zs1.reshape(-1,1),Zs2.reshape(-1,1)),dim=1)

transfunc = SparseGP(kern, Zs) # choose sparse GP as a transition function
transfunc.q_mu.data = torch.randn(transfunc.q_mu.size()).type(torch.float64) # random initialisation for inducing point posterior mean

# uncomment to instead condition on fixed points
#Zs_fx = torch.tensor([0., 0.]).view(-1,xDim)
# transition function conditioned on fixed point
#transfunc = SparseGP(kern, Zs, Zs_fx) 

d:\logiciel\miniconda\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


# Build the model

In [7]:
#Build the model

# build point procces generative model for continuous time spike-time observations
model = PointProcessGPSDEmodel(xDim, transfunc, outputMapping, like, nLeg=100)

# assemble inference algorithm
inference = GaussMarkovLagrange(xDim, trLen, learningRate=1, dtstep=dtgrid)

# create GPSDE model object (final object)
myGPSDE = GPSDE(model, inference)

# fix inducing points on chosen grid
myGPSDE.model.transfunc.Zs.requires_grad = False

# Testing how long it takes to run the algorithm

estep number of step for inference, mstepiter for learning update, niter for everything

We first try to do only one step for every part for the all dataset (niter=1,estepIter=1,mstepiter=1) => more than 130 min and it was stopped in inference update

In [28]:
myGPSDE.variationalEM(niter=1,eStepIter=1, mStepIter=1) #estep number of step for inference, mstepiter for learning update

c:\Users\fosmo\OneDrive\Documents\GitHub\gp-sde\demos\..\kernels.py:320: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  ..\aten\src\ATen\native\BatchLinearAlgebra.cpp:766.)
  scaled_diffs, _ = torch.solve(mu_x2_new_diffs.transpose(-1, -2), lengthscales_new_squared.unsqueeze(-3))


KeyboardInterrupt: 

We want to see on a smaller subset of data which part of the algorithm takes longer

On 3 neurons with 1 general iteration, 1 iteration for inference and 1 iteration for learning:

inference time = 298 s, learning time= 103 s ,update time =0s, Total : 13 min
Details inference :
forward= 35s, grad= 0.1 s, backward=277s, update= 1.7

In [8]:
N_neurons=3 #only 3 neurons
Y_spike_small=[Yspike[0][:N_neurons]]
print(len(Y_spike_small[0]))

like_small = PointProcess(Y_spike_small, link, trLen, dtstep=dtgrid, nLeg=100) # point process likelihood 
C_small = 2.*np.random.rand(xDim,N_neurons) * np.random.choice([-1,1],size=(xDim,N_neurons)) 
d_small = 0.1*np.random.randn(1,N_neurons)
outputMapping_small = AffineMapping(torch.tensor(C_small), torch.tensor(d_small)) # affine output mapping, used inn the forward of the model
model_small = PointProcessGPSDEmodel(xDim, transfunc, outputMapping_small, like_small, nLeg=100)
# create GPSDE model object (final object)
GPSDEsmall = GPSDE(model_small, inference)
# fix inducing points on chosen grid
GPSDEsmall.model.transfunc.Zs.requires_grad = False

3


In [9]:
GPSDEsmall.variationalEM(niter=1,eStepIter=1, mStepIter=1)

timeforward34.89074230194092
time grad0.1396312713623047


c:\Users\fosmo\OneDrive\Documents\GitHub\gp-sde\demos\..\kernels.py:320: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  ..\aten\src\ATen\native\BatchLinearAlgebra.cpp:766.)
  scaled_diffs, _ = torch.solve(mu_x2_new_diffs.transpose(-1, -2), lengthscales_new_squared.unsqueeze(-3))


time backward277.1971700191498
time update infer1.6904823780059814
inference time=313.92301058769226


c:\Users\fosmo\OneDrive\Documents\GitHub\gp-sde\demos\..\utils.py:140: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).transpose(-2, -1).conj().
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at  ..\aten\src\ATen\native\BatchLinearAlgebra.cpp:1285.)
  L = torch.cholesky(M)


learning time=86.98113107681274
update time0.0009975433349609375
-------------------------------------------------------
iter   objective    log-like      kl-div     f-prior
-------------------------------------------------------
   0    3911.546   -2675.130     343.637    -892.778
timeforward34.368138551712036
time grad0.13862824440002441


KeyboardInterrupt: 